In [1]:
import sys
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
sys.path.append('/workspace/wilson/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from model_utils import *
from group_utils import *
from group_data import *
from jaxtyping import Float
from typing import Union
from einops import repeat
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
from llc import *
import gc
from collections import defaultdict

%load_ext autoreload
%autoreload 2

In [2]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [3]:
path = '/workspace/models/2024-08-22_00-54-00_SL2_5_MLP3_512'
models, params = load_models(path, sel='final')

In [4]:
data = GroupData(params)

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: smallgrp(120, 5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


In [86]:
loss_dict = test_loss(models[-1].to(device), data)
#instance = loss_dict['G0_acc'].argmax().item()
instance = 42
model = models[0][instance]
instance

42

In [87]:
loss_dict = test_loss(model.to(device), data)

In [88]:
loss_dict

{'G0_loss': tensor([0.0731], device='cuda:0'),
 'G0_acc': tensor([0.9999], device='cuda:0')}

In [89]:
test_inputs = t.tensor(list(product(range(120), repeat=2)), device=device)

In [90]:
labels = einops.rearrange(data.groups[0].cayley_table, "a b -> (a b)").to(device)

In [91]:
out = model(test_inputs)

In [84]:
t.softmax(out[948,0], dim=0)[117], t.softmax(out[948,0], dim=0)[97]

(tensor(0.4049, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(0.4725, device='cuda:0', grad_fn=<SelectBackward0>))

In [92]:
out = out.argmax(dim=-1).flatten()

In [93]:
(out == labels).sum()

tensor(14399, device='cuda:0')

In [96]:
wrong = (out != labels).nonzero().item()
wrong

5996

In [97]:
test_inputs[wrong], out[wrong], labels[wrong]

(tensor([ 49, 116], device='cuda:0'),
 tensor(57, device='cuda:0'),
 tensor(82, device='cuda:0'))

In [98]:
gap.Image(iso, g.Elements()[49])

[ [ 0*Z(5), Z(5) ], [ Z(5), Z(5)^2 ] ]

In [99]:
gap.Image(iso, g.Elements()[116])

[ [ Z(5)^0, Z(5)^2 ], [ Z(5)^2, Z(5) ] ]

In [44]:
948 // 120, 948 % 120

(7, 108)

In [45]:
data.groups[0].elements[7]

'( 1, 2, 4, 8)( 3,17, 9,12)( 5,15, 6,14)( 7,19,13,18)(10,16,11,21)(20,23,24,22)'

In [46]:
data.groups[0].elements[108]

'( 1,22,16, 4,23,21)( 2,24,18, 8,20,19)( 3,12,10, 9,17,11)( 5,14,13, 6,15, 7)'

In [53]:
g = data.groups[0].gap_repr

In [54]:
g.IsSL()

true

In [51]:
sl = gap.SL(2, 5)

In [55]:
iso = gap.IsomorphismGroups(g, sl)

In [58]:
gap.Image(iso, g.Elements()[7])

[ [ Z(5), 0*Z(5) ], [ Z(5), Z(5)^3 ] ]

In [59]:
gap.Image(iso, g.Elements()[108])

[ [ 0*Z(5), Z(5) ], [ Z(5), Z(5)^0 ] ]

In [63]:
gap.Image(iso, g.Elements()[117])

[ [ 0*Z(5), Z(5)^2 ], [ Z(5)^0, Z(5) ] ]

In [76]:
gap.Image(iso, g.Elements()[97])

[ [ 0*Z(5), Z(5)^0 ], [ Z(5)^2, Z(5)^3 ] ]

In [68]:
labels[948]

tensor(117, device='cuda:0')

In [69]:
out[948]

tensor(97, device='cuda:0')

In [61]:
test_inputs[948]

tensor([  7, 108], device='cuda:0')

In [70]:
data.groups[0].elements[7]

'( 1, 2, 4, 8)( 3,17, 9,12)( 5,15, 6,14)( 7,19,13,18)(10,16,11,21)(20,23,24,22)'

In [71]:
g.Elements()[7]

(1,2,4,8)(3,17,9,12)(5,15,6,14)(7,19,13,18)(10,16,11,21)(20,23,24,22)

In [72]:
g.Elements()[7] * g.Elements()[108]

(1,24,16,3,11)(2,23,18,5,7)(4,20,21,9,10)(6,13,8,22,19)

In [74]:
g.Elements()[117]

(1,24,16,3,11)(2,23,18,5,7)(4,20,21,9,10)(6,13,8,22,19)

In [75]:
gap.eval('Print(Z(5));')

In [78]:
full_train_loss(model, data)

tensor([0.0760], device='cuda:0', grad_fn=<DivBackward0>)